Revised and fixed code from https://github.com/JayParks/transformer (MT) for LM

Stripped the code from the JayParks repo for MT Transformer. Introduced a few updates and changes for speed, but it's still frustratingly slow. Possible improvement - speed it up.

Another issue - hyperparameter search for language modelling (number of heads, number of self-attention layers, etc). Does not work well from the box. This might be of help https://arxiv.org/pdf/1804.00247.pdf.

Also consider parallelizing.

# TODO

* Clean up
* Add MoS

# Sentence-wise batching

This version of Transformer LM uses sentence-wise batching (each sentence is an inpdependent example).

**NB** Before running make sure src code accounts for PAD.

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm

import torch
torch.cuda.device(0)
import torch.nn as nn
import torch.optim as optim

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as pad
from torch.nn.utils import clip_grad_norm_ as clip
from torch.optim.lr_scheduler import StepLR

import const
from data import *
from transformer import *

In [3]:
ptb_datapath_train = 'data/penn/train.txt'
ptb_datapath_valid = 'data/penn/valid.txt'
ptb_datapath_test = 'data/penn/test.txt'

batch_size = 128

ptb_train = DataSet(ptb_datapath_train, batch_size, display_freq=0, max_len=90, trunc_len=90)
ptb_valid = DataSet(ptb_datapath_valid, batch_size, display_freq=0, max_len=90, trunc_len=90)
ptb_test = DataSet(ptb_datapath_test, batch_size, display_freq=0, max_len=90, trunc_len=90)

Loading data from data/penn/train.txt ...
Loading data from data/penn/valid.txt ...
Loading data from data/penn/test.txt ...


In [4]:
ptb_train.build_dict()
ptb_valid.change_dict(ptb_train.dictionary)
ptb_test.change_dict(ptb_train.dictionary)

Building dictionary...
Done.
Save dictionary at data/penn/train.txt.dict
Index tokens ...
42068 sentences were processed, 0 longer than maximum length,0 were ignored because zero length
Data discription:
Data name : data/penn/train.txt
Number of sentence : 42068
Number of tokens : 887521
Vocabulary size : 10000
Number of batches : 328
Batch size : 128
Done.
Index tokens ...
3370 sentences were processed, 0 longer than maximum length,0 were ignored because zero length
Data discription:
Data name : data/penn/valid.txt
Number of sentence : 3370
Number of tokens : 70390
Vocabulary size : 10000
Number of batches : 26
Batch size : 128
Done.
Index tokens ...
3761 sentences were processed, 0 longer than maximum length,0 were ignored because zero length
Data discription:
Data name : data/penn/test.txt
Number of sentence : 3761
Number of tokens : 78669
Vocabulary size : 10000
Number of batches : 29
Batch size : 128
Done.


In [5]:
# ############ Optional: get data by tokens ###############
# corpus = Corpus('data/penn')
# eval_batch_size = 10
# test_batch_size = 1
# batch_size = 128
# train_data = batchify(corpus.train, batch_size)
# val_data = batchify(corpus.valid, eval_batch_size)
# test_data = batchify(corpus.test, test_batch_size)

# #### how to take a batch ####
# # the data is already splitten into batch_size(now we need to decide about seq length)
# batch_num = 2
# batch = get_batch(train_data, batch_num, seq_len=35)


# #### TODO (if needed) ###
# # 1) repackage hiddens for learning by tokens
# # 2) learn not every step (depends on 1st point)
# # 3) add grad clipping

In [6]:
voc_size = ptb_train.num_vocb
emb_dim = 512
d_k = 64
d_v = 64
n_layers = 2
n_heads = 4
d_ff = 2048
max_tgt_seq_len = 90
dropout = 0.1
weighted_model = False
share_proj_weight = True
lr = 1e-6
n_epochs = 10
clip_grad = 5
warmup_steps = 2000

In [7]:
model = LMTransformer(n_layers, d_k, d_v, emb_dim, d_ff,
                      n_heads, max_tgt_seq_len, voc_size,
                      dropout, weighted_model, share_proj_weight)
criterion = nn.CrossEntropyLoss(ignore_index=const.PAD)

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

#opt = optim.Adam(model.trainable_params(), lr=lr)
# lr_lambda = lambda epoch: 0.99 ** epoch
#lrsched = StepLR(opt, step_size=10, gamma=0.5)

Sharing target embedding and projection..


In [8]:
torch.cuda.is_available()

True

In [9]:
opt = optim.Adam(model.trainable_params(),betas=(0.9, 0.98), eps=1e-09, lr=lr)
i=0
for epoch in range(n_epochs):
    #lrsched.step()
    acc_loss = 0
    print('Start epoch %d, learning rate %f '%(epoch + 1, opt.state_dict()['param_groups'][0]['lr']))
    start_time = time.time()
    model.train()
    ptb_train.shuffle()
    for batch_idx in tqdm(range(ptb_train.num_batch), unit='batches'):
        data, lengths, target = ptb_train.get_batch(batch_idx)
        opt.zero_grad()
        output, self_attn = model.forward(data, lengths)
        break
    break
#         loss = criterion(output, target.view(-1))
#         loss.backward()
#         opt.step()
#         acc_loss += loss.item()
#         i+=1
#         new_lr = np.power(emb_dim, -0.5) * np.min([
#             np.power((i), -0.5),
#             np.power(warmup_steps, -1.5) * (i)])
#         for param_group in opt.param_groups:
#             param_group['lr'] = new_lr
        
#     avg_loss = acc_loss / ptb_train.num_batch
#     print('Epoch : %d, Batch : %d / %d, Loss : %f, Perplexity : %f, Time : %f' 
#           % (epoch + 1, batch_idx, ptb_train.num_batch,
#              avg_loss, math.exp(avg_loss),
#              time.time() - start_time))

#     acc_loss = 0
#     model.eval()
#     for batch_idx in tqdm(range(ptb_test.num_batch), unit='batches'):
#         data, lengths, target = ptb_test[batch_idx]
#         output, self_attn = model.forward(data, lengths)
#         loss = criterion(output, target.view(-1))
#         acc_loss += loss.item()

#     val_loss = acc_loss / ptb_test.num_batch
#     print('Validation Loss : %f' % val_loss)
#     print('Validation Perplexity : %f' % math.exp(val_loss))

  0%|          | 0/328 [00:00<?, ?batches/s]

Start epoch 1, learning rate 0.000001 
2
emb
torch.Size([128, 53, 512])
tensor([[-1.3810e-01,  7.9967e-02, -8.8474e-01,  ..., -5.7051e-02,
         -4.4142e-02, -9.6475e-01],
        [ 1.0865e+00, -1.1131e+00,  8.1322e-01,  ...,  6.8507e-01,
          3.6231e-01, -1.7098e-01],
        [-5.3852e-01,  8.0949e-01, -2.0556e-01,  ...,  6.1222e-01,
         -6.3589e-01, -8.0007e-01],
        ...,
        [-3.7327e-02, -6.5427e-01, -1.6921e+00,  ..., -6.6768e-01,
          9.1436e-04, -1.0795e-01],
        [-5.0567e-02, -7.0362e-01,  2.4117e+00,  ...,  6.2272e-01,
         -1.8694e+00,  2.4068e-01],
        [-4.7632e-01,  1.7441e+00, -5.9936e-02,  ...,  7.9750e-01,
          7.0828e-01,  2.1999e+00]], device='cuda:0')
pos emb
torch.Size([128, 53, 512])
tensor([[-1.3810e-01,  1.0800e+00, -8.8474e-01,  ...,  9.4295e-01,
         -4.4142e-02,  3.5245e-02],
        [ 1.9280e+00, -5.7280e-01,  1.6351e+00,  ...,  1.6851e+00,
          3.6241e-01,  8.2902e-01],
        [ 3.7078e-01,  3.9334e-01,  7.